In [3]:
import pandas as pd

In [4]:
df = pd.read_csv("products.csv")
df.head()

,timestamp,shoe_id,shoe_type,shoe_size,shoe_color,shoe_brand,price,popularity,durability
0,2025-03-18 10:04:33,1,sneakers,7,red,reebok,70,3,2
1,2025-03-18 10:04:32,2,sneakers,9,blue,adidas,100,3,5
2,2025-03-18 10:04:31,3,boots,8,brown,dr martens,100,2,1
3,2025-03-18 10:04:30,4,boots,8,brown,dr martens,180,3,5
4,2025-03-18 10:04:29,5,sneakers,11,red,new balance,60,2,1


In [5]:
from sklearn.neighbors import NearestNeighbors

In [6]:
# Load the data
data = pd.read_csv("products.csv")

# Drop the datetime column if it exists
if 'timestamp' in data.columns:
    data = data.drop(columns=['timestamp'])

# Use get_dummies to encode categorical variables
data_encoded = pd.get_dummies(data, columns=['shoe_type', 'shoe_color', 'shoe_brand'])

# Features for the model
features = data_encoded.columns
X = data_encoded

In [7]:
# Initialize the NearestNeighbors model
model = NearestNeighbors(n_neighbors=5, algorithm='auto')

# Fit the model
model.fit(X)


NearestNeighbors()

In [26]:
from flask import Flask, request, jsonify

app = Flask(__name__)

@app.route('/recommend', methods=['POST'])
def recommend():
    try:
        # Get the JSON data from the POST request
        current_purchases = request.json

        # Create a DataFrame for the current purchases
        current_purchases_df = pd.DataFrame(current_purchases)

        # Use get_dummies to encode the current purchases
        current_purchases_encoded = pd.get_dummies(current_purchases_df, columns=['shoe_type', 'shoe_color', 'shoe_brand'])

        # Align the current purchases with the training data
        current_purchases_encoded = current_purchases_encoded.reindex(columns=features, fill_value=0)

        # Calculate the average feature values across all purchases
        average_purchase_encoded = current_purchases_encoded.mean(axis=0).to_frame().T

        # Find the nearest neighbors for the averaged purchase
        distances, indices = model.kneighbors(average_purchase_encoded)

        # Get the top 5 recommended products
        recommended_products = data.iloc[indices[0]]

        # Convert the recommended products to a JSON response
        return jsonify(recommended_products.to_dict(orient='records'))

    except Exception as e:
        return jsonify({'error': str(e)}), 400

if __name__ == '__main__':
    app.run(debug=True, port=5000)

 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
 * Restarting with stat


SystemExit: 1

C:\Users\wowom\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\IPython\core\interactiveshell.py:3557: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
